In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, MultiHeadAttention, LayerNormalization, Add, Conv1D, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
# 1. 데이터 로드 및 전처리
# 파일 경로

pollutants_path = '../../NewData/Weekly_Air_Pollutants.csv'
greenhouse_gas_path = '../../NewData/Weekly_Greenhouse_Gas.csv'
population_path = '../../NewData/Weekly_Population.csv'
power_usage_path = '../../NewData/Weekly_Power_Usage.csv'
temperature_path = '../../ClimateDataTeam/climate_data/merged_weekly_avg_temp.csv'
target_path = population_path

# 데이터 로드
pollutants_df = pd.read_csv(target_path)
temperature_df = pd.read_csv(temperature_path)

# 속성값 정의
pollutants_column = pollutants_df.columns.tolist()
climate_column = temperature_df.columns.tolist()

# datetime 컬럼 변환 및 병합
pollutants_df['datetime'] = pd.to_datetime(pollutants_df['datetime'])
temperature_df['datetime'] = pd.to_datetime(temperature_df['datetime'])
merged_df = pd.merge(pollutants_df, temperature_df, on='datetime', how='inner')

# datetime 형변환 및 제거
merged_df['year'] = merged_df['datetime'].dt.year
merged_df['month'] = merged_df['datetime'].dt.month
merged_df['week'] = merged_df['datetime'].dt.isocalendar().week
merged_df = merged_df.drop(columns=['datetime'])  # datetime 제거

# pollutants_column에 날짜데이터 추가
pollutants_column.append('year')
pollutants_column.append('month')
pollutants_column.append('week')
pollutants_column = pollutants_column[1:]
climate_column = climate_column[1:]

print(f"pollutants_column : {pollutants_column}")
print(f"climate_column : {climate_column}")

pollutants_column : ['TotalPopulation', 'SeoulPopulation', 'BusanPopulation', 'DaeguPopulation', 'InchenPopulation', 'GwanguPopulation', 'DaejeonPopulation', 'UlsanPopulation', 'GyeonggiPopulation', 'GangwonPopulation', 'ChungcheongbukPopulation', 'ChungcheongnamPopulation', 'JeonbukPopulation', 'JeollanamPopulation', 'GyeongsangbukPopulation', 'GyeongsangnamPopulation', 'JejuPopulation', 'year', 'month', 'week']
climate_column : ['tempmax', 'tempmin', 'temp', 'dew', 'humidity', 'precip', 'windspeed', 'sealevelpressure', 'moonphase']


In [3]:
# X와 y 분리
X = merged_df.drop(columns=pollutants_column[:])  # datetime 및 출력 변수 제외
y = merged_df[climate_column[:]]  # 출력 변수

In [4]:
# 2. 시계열 윈도우 생성 함수
def create_time_series_features(X, y, lag):
    X_features, y_labels = [], []
    for i in range(len(X) - lag):
        X_features.append(X.iloc[i:i+lag].values)
        y_labels.append(y.iloc[i+lag].values)
    return np.array(X_features), np.array(y_labels)

# 시계열 윈도우 생성
lag = 30  # 과거 10주 데이터를 사용
X_ts, y_ts = create_time_series_features(X, y, lag)

# 데이터 확인
print(f"X_ts shape: {X_ts.shape}, y_ts shape: {y_ts.shape}")

X_ts shape: (1021, 30, 9), y_ts shape: (1021, 9)


In [ ]:
# 3. Transformer 블록 정의
def transformer_block(x, num_heads, key_dim, ff_dim, dropout_rate):
    # Multi-Head Attention
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(x, x)
    attn_output = Dropout(dropout_rate)(attn_output)
    out1 = Add()([x, attn_output])
    out1 = LayerNormalization(epsilon=1e-6)(out1)

    # Feed-forward Network
    ffn_output = Dense(ff_dim, activation="relu")(out1)  # ff_dim이 out1의 feature 크기와 동일하게 설정
    ffn_output = Dropout(dropout_rate)(ffn_output)

    # Shape 맞추기 (이 중 하나 선택)
    # 1. ff_dim을 out1과 동일하게 맞춤
    # ffn_output = Dense(out1.shape[-1], activation="relu")(out1)

    # 2. out1의 feature 크기를 ff_dim으로 확장
    out1_resized = Dense(ff_dim)(out1)
    out2 = Add()([out1_resized, ffn_output])

    # Layer Normalization
    out2 = LayerNormalization(epsilon=1e-6)(out2)
    return out2

In [6]:
# 4. 모델 생성
def create_model(input_shape, output_dim, num_heads=4, key_dim=32, ff_dim=128, dropout_rate=0.1):
    inputs = Input(shape=input_shape)

    # Transformer Block
    x = transformer_block(inputs, num_heads=num_heads, key_dim=key_dim, ff_dim=ff_dim, dropout_rate=dropout_rate)

    # Flatten Transformer Output
    x = GlobalAveragePooling1D()(x)  # time_steps 축 제거
    x = Dropout(0.2)(x)

    # Output Layer
    outputs = Dense(output_dim)(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=['mae'])
    return model

In [7]:
# 5. 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_ts, y_ts, test_size=0.2, random_state=42)

In [8]:
# 6. 모델 정의 및 학습
# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',  # 기준으로 삼을 값 (예: 검증 손실)
    patience=10,          # 몇 epoch 동안 개선이 없으면 중단
    restore_best_weights=True  # 가장 성능이 좋았던 가중치를 복원
)

input_shape = (X_train.shape[1], X_train.shape[2])  # (lag, 입력 특성 수)
output_dim = y_ts.shape[1]  # 출력 변수 수
model = create_model(input_shape, output_dim)

In [9]:
# 모델 학습
model.fit(X_train, y_train, 
          epochs=1000, 
          batch_size=32, 
          validation_split=0.2,
          callbacks=[early_stopping]
          )

Epoch 1/1000
21/21 [==============================] - 2s 14ms/step - loss: 116566.4922 - mae: 144.1053 - val_loss: 116451.4297 - val_mae: 143.5321
Epoch 2/1000
21/21 [==============================] - 0s 6ms/step - loss: 116338.3281 - mae: 143.8385 - val_loss: 116198.3750 - val_mae: 143.2543
Epoch 3/1000
21/21 [==============================] - 0s 6ms/step - loss: 116067.7031 - mae: 143.5723 - val_loss: 115940.5625 - val_mae: 142.9753
Epoch 4/1000
21/21 [==============================] - 0s 6ms/step - loss: 115823.9375 - mae: 143.3071 - val_loss: 115677.2656 - val_mae: 142.6834
Epoch 5/1000
21/21 [==============================] - 0s 5ms/step - loss: 115571.9375 - mae: 142.9986 - val_loss: 115421.2109 - val_mae: 142.3756
Epoch 6/1000
21/21 [==============================] - 0s 5ms/step - loss: 115325.8125 - mae: 142.7426 - val_loss: 115165.6797 - val_mae: 142.0689
Epoch 7/1000
21/21 [==============================] - 0s 6ms/step - loss: 115065.8984 - mae: 142.4465 - val_loss: 114896.19

In [10]:
# 7. 예측 및 평가
y_pred = model.predict(X_test)

# 평가
mae = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')

print(f"MAE per output variable: {mae}")
print(f"MSE per output variable: {mse}")

7/7 [==============================] - 0s 2ms/step
MAE per output variable: [16.43397203 17.14617711 16.49291552 17.64283885  7.87863013  0.24691809
  1.4762638   6.59444571  0.18660482]
MSE per output variable: [3.45547930e+02 3.82255740e+02 3.50146991e+02 4.10507296e+02
 9.72270643e+01 2.02524539e-01 3.46021232e+00 5.79124177e+01
 4.84902210e-02]
